In [ ]:
import pandas as pd

In [ ]:
dbcamhd = pd.read_json('dbcamhd.json', orient="records", lines=True)

#### Define the frame numbers from each file to process

In [ ]:
blob_urls = list(dbcamhd.blob_url[(dbcamhd.deployment == 2) & (dbcamhd.frame_count > 5000) & (dbcamhd.frame_count < 30000)])
blob_urls.sort()

In [ ]:
frame_numbers = [3841, 3933, 4052, 4171, 4263, 4382]

In [ ]:
import pycamhd as camhd
import numpy as np
from dask import delayed
import dask.array as dsa

In [ ]:
delayed_frame_list = []
for blob_url in blob_urls:
    delayed_moov_atom = delayed(camhd.get_moov_atom)(blob_url)         
    for frame_number in frame_numbers:
        delayed_frame = delayed(camhd.get_frame)(blob_url, frame_number, 'gray16le', delayed_moov_atom)
        delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))

In [ ]:
delayed_frame_array = dsa.stack(delayed_frame_list)
delayed_frame_array

# 2) Load a subset of images as Dask array